In [ ]:
# %pip install implicit

In [8]:
# %pip install -U gensim

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import pandas as pd
import numpy as np

import pickle
from tqdm import tqdm
import gc
from pathlib import Path

import warnings
import sys
from IPython.core.interactiveshell import InteractiveShell

warnings.filterwarnings("ignore")
sys.path.append("../") # path to the `src`` folder
InteractiveShell.ast_node_interactivity = "all"
tqdm.pandas()

In [10]:
from gensim.models import Word2Vec
# from gensim.models import KeyedVectors

In [2]:
from src.data import DataHelper
import pickle

In [4]:
data_dir = Path("../data/")
model_dir = Path("../models/")

In [5]:
dh = DataHelper(data_dir)
# data = dh.preprocess_data(save=True, name="encoded_full") # * run only once, processed data will be saved
data = dh.load_data(name="encoded_full")

### Train


In [6]:
inter = pd.read_parquet(data_dir / "processed/processed_inter.pqt")
data["inter"] = inter

In [7]:
inter['t_dat'] = pd.to_datetime(inter['t_dat'])
last_week_start = pd.to_datetime("2020-08-19")
inter = inter.loc[(inter.t_dat < last_week_start)]

In [13]:
feedid_seq_list = inter.groupby(['customer_id']).article_id.apply(lambda x: [str(id) for id in x] ).values
model_sg = Word2Vec(feedid_seq_list,  vector_size=64, window=32, min_count=1, sg=0, sample=1e-3, negative=15, workers=32, seed=1, epochs=10)
model_sg.save(open(model_dir/'articleid_model_cbow.model','wb'))

In [14]:
model_sg = pickle.load(open(model_dir/'articleid_model_cbow.model','rb'))

In [ ]:
# * Customer Embedding
feedid_seq_list = inter.groupby(['customer_id']).article_id.apply(lambda x: [str(id) for id in x] ).reset_index()

In [17]:
full_users = data['user']['customer_id'].values
customer_embedding = np.ones((len(full_users)+1, 64))/64

for uid, items in tqdm(feedid_seq_list.values):
    if len(items)>1:
        vec = np.mean(list(map(lambda x: model_sg.wv[x], items)), axis=0)
    else:
        vec = model_sg.wv[items[0]]
    customer_embedding[uid] = vec/np.sqrt(np.sum(vec**2))

100%|██████████| 1356709/1356709 [01:52<00:00, 12049.90it/s]


In [18]:
del full_users, feedid_seq_list, inter
gc.collect()

1264

In [19]:
customer_embedding.dump(data_dir/'external'/'w2v_user_embd.npy')

In [20]:
# * Article Embedding
full_items = data['item']['article_id'].values
article_embedding = np.ones((len(full_items)+1, 64))/64
for item in tqdm(full_items):
    try:
        vec = model_sg.wv[str(item)]
    except:
        vec = article_embedding[item]
    article_embedding[item,:] = vec/np.sqrt(np.sum(vec**2))

100%|██████████| 105542/105542 [00:01<00:00, 53860.10it/s]


In [21]:
article_embedding.dump(data_dir/'external'/'w2v_item_embd.npy')